In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define the tickers and download the data
tickers = ["ALL", "ASGN", "CI", "COP", "EME", "EVR", "GILD", "GPK", 
           "ISRG", "MKL", "MOH", "PEG", "PXD", "QCOM", "UBSI", "VFC", "XEL"]

# Download the adjusted close prices for the specified time period
start_date = "2017-11-27"
end_date = "2019-11-27"
data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

# Initialize dataframes for results
ror = pd.DataFrame(index=tickers, columns=['BandH', 'MACD', 'RSI'])
risk = pd.DataFrame(index=tickers, columns=['BandH', 'MACD', 'RSI'])
sharpe_ratio = pd.DataFrame(index=tickers, columns=['BandH', 'MACD', 'RSI'])

for ticker in tickers:
    # Calculate MACD and Signal Line
    data['MACD'] = data[ticker].ewm(span=12, adjust=False).mean() - data[ticker].ewm(span=26, adjust=False).mean()
    data['Signal_Line'] = data['MACD'].ewm(span=9, adjust=False).mean()

    # Calculate RSI
    delta = data[ticker].diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    data['RSI'] = 100 - (100 / (1 + rs))

    # Buy and Hold
    data['BandH_Returns'] = data[ticker].pct_change().fillna(0)

    # MACD
    data['MACD_Returns'] = np.where(data['MACD'] > data['Signal_Line'], data[ticker].pct_change(), 0)

    # RSI
    data['RSI_Returns'] = np.where(data['RSI'] < 30, data[ticker].pct_change(), 
                                   np.where(data['RSI'] > 70, -data[ticker].pct_change(), 0))

    # Store metrics
    ror.loc[ticker] = [data['BandH_Returns'].sum(), data['MACD_Returns'].sum(), data['RSI_Returns'].sum()]
    risk.loc[ticker] = [data['BandH_Returns'].std(), data['MACD_Returns'].std(), data['RSI_Returns'].std()]
    sharpe_ratio.loc[ticker] = ror.loc[ticker] - 0.025 / risk.loc[ticker]

# Print results
print("Rate of Return:\n", ror)
print("\nRisk:\n", risk)
print("\nSharpe Ratio:\n", sharpe_ratio)


[*********************100%%**********************]  17 of 17 completed
Rate of Return:
          BandH      MACD       RSI
ALL   0.199301  0.498367 -0.595125
ASGN  0.161419  1.132448 -1.455774
CI    0.084516   0.63507 -0.939567
COP   0.323553   0.96389 -1.010668
EME   0.206767  0.756186 -0.867812
EVR   0.054431  0.755758 -1.172998
GILD  0.064041  0.494842 -0.532873
GPK   0.193111  0.815494 -1.027605
ISRG  0.470455   0.87648 -0.894701
MKL    0.08292  0.314591 -0.436752
MOH   0.721575  1.323886  -1.68478
PEG   0.236139  0.511065 -0.392762
PXD  -0.042432  0.904587 -1.119372
QCOM  0.411709  1.318585 -1.955371
UBSI  0.212503  0.677338 -0.880774
VFC   0.353037  0.838534  -0.92889
XEL   0.272316   0.50692 -0.684659

Risk:
          BandH      MACD       RSI
ALL   0.011881  0.007019  0.005915
ASGN   0.02024  0.014647  0.011356
CI    0.017965  0.012126   0.01048
COP   0.018122  0.012117  0.009673
EME   0.014503  0.009084  0.007389
EVR   0.018372  0.012641   0.01035
GILD   0.01557  0.009764  0.0

In [ ]:
print(ror["BandH"].mean())
print(ror["RSI"].mean())
print(ror["MACD"].mean())

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define the tickers and download the data
tickers = ["ALL", "ASGN", "CI", "COP", "EME", "EVR", "GILD", "GPK", 
           "ISRG", "MKL", "MOH", "PEG", "PXD", "QCOM", "UBSI", "VFC", "XEL"]

# Download the adjusted close prices for the specified time period
start_date = "2017-11-27"
end_date = "2019-11-27"
data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

# MACD strategy
def compute_macd_strategy(data):
    short_window = 12
    long_window = 26
    signal_window = 9
    
    data['12_ema'] = data['Adj Close'].ewm(span=short_window, adjust=False).mean()
    data['26_ema'] = data['Adj Close'].ewm(span=long_window, adjust=False).mean()
    data['macd'] = data['12_ema'] - data['26_ema']
    data['signal'] = data['macd'].ewm(span=signal_window, adjust=False).mean()
    
    data['macd_signal'] = np.where(data['macd'] > data['signal'], 1, -1)
    
    return data

# RSI strategy
def compute_rsi_strategy(data):
    rsi_period = 14
    
    delta = data['Adj Close'].diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)
    avg_gain = gain.rolling(window=rsi_period).mean()
    avg_loss = loss.rolling(window=rsi_period).mean()
    rs = avg_gain / avg_loss
    data['rsi'] = 100 - (100 / (1 + rs))
    
    data['rsi_signal'] = np.where(data['rsi'] < 30, 1, np.where(data['rsi'] > 70, -1, 0))
    
    return data

# Buy and Hold strategy
def compute_bandh_strategy(data):
    data['bandh_signal'] = 1
    return data


# Calculate Rate of Return, Risk, and Sharpe Ratio
def compute_performance_metrics(data, strategy_column, risk_free_rate=2.5):
    transaction_cost = 0.00025

    data['position'] = data[strategy_column].shift(1)
    data['entry'] = np.where(data['position'] == 1, data['Adj Close'], np.nan)
    data['exit'] = np.where(data['position'] == -1, data['Adj Close'], np.nan)
    data['entry'].ffill(inplace=True)
    data['returns'] = np.where(data['position'] == -1, 
                               (data['exit'] - (data['entry'] * (1 + transaction_cost))) / data['entry'], 
                               np.nan)
    returns = np.array(data['returns'])
    returns_only = returns[~np.isnan(returns)]

    rate_of_return = sum(returns_only)
    risk = returns_only.std()
    sharpe_ratio = (rate_of_return - risk_free_rate) / risk
    
    return rate_of_return, risk, sharpe_ratio

# Apply strategies and compute metrics
for ticker in tickers:
    df = data[[ticker]].rename(columns={ticker: 'Adj Close'})
    
    df_macd = compute_macd_strategy(df.copy())
    df_rsi = compute_rsi_strategy(df.copy())
    df_bandh = compute_bandh_strategy(df.copy())
    
    macd_return, macd_risk, macd_sharpe = compute_performance_metrics(df_macd, 'macd_signal')
    rsi_return, rsi_risk, rsi_sharpe = compute_performance_metrics(df_rsi, 'rsi_signal')
    bandh_return, bandh_risk, bandh_sharpe = compute_performance_metrics(df_bandh, 'bandh_signal')
    
    print(f"Metrics for {ticker}:")
    print(f"MACD - Rate of Return: {macd_return:.2f}, Risk: {macd_risk:.2f}, Sharpe Ratio: {macd_sharpe:.2f}")
    print(f"RSI - Rate of Return: {rsi_return:.2f}, Risk: {rsi_risk:.2f}, Sharpe Ratio: {rsi_sharpe:.2f}")
    print(f"Buy and Hold - Rate of Return: {bandh_return:.2f}, Risk: {bandh_risk:.2f}, Sharpe Ratio: {bandh_sharpe:.2f}")
    print("-" * 50)


In [ ]:
import pandas as pd

def compute_rsi(data, window):
    """
    Compute the RSI (Relative Strength Index) of a pandas DataFrame.
    
    Parameters:
    - data (pd.Series): A pandas Series containing the price data.
    - window (int): The period for which RSI should be calculated.
    
    Returns:
    - pd.Series: A pandas Series containing the RSI values.
    """
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)

    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

# Sample data
data = pd.Series([44.34, 44.09, 44.15, 43.61, 44.33, 44.83, 45.10, 45.42, 
                  45.84, 46.08, 45.89, 46.03, 45.61, 46.28, 46.28, 46.00, 
                  46.03, 46.41, 46.22, 46.29, 46.53, 46.48, 46.46, 46.23, 
                  46.08, 46.03, 45.61, 45.00, 45.03, 44.23, 43.95, 43.63])

rsi = compute_rsi(data, 14)
print(rsi)

In [ ]:
import pandas as pd
import numpy as np
import ta
import yfinance as yf

def compute_rsi(data, window):
    """
    Compute the RSI (Relative Strength Index) of a pandas DataFrame.
    
    Parameters:
    - data (pd.Series): A pandas Series containing the price data.
    - window (int): The period for which RSI should be calculated.
    
    Returns:
    - pd.Series: A pandas Series containing the RSI values.
    """
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)

    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

def compute_macd(data, short_window=12, long_window=26, signal_window=9):
    """
    Compute the MACD (Moving Average Convergence Divergence) of a pandas DataFrame.
    
    Parameters:
    - data (pd.Series): A pandas Series containing the price data.
    - short_window (int): The short period for EMA. Default is 12.
    - long_window (int): The long period for EMA. Default is 26.
    - signal_window (int): The signal period for EMA. Default is 9.
    
    Returns:
    - pd.DataFrame: A pandas DataFrame containing the MACD and Signal values.
    """
    short_ema = data.ewm(span=short_window, adjust=False).mean()
    long_ema = data.ewm(span=long_window, adjust=False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()

    return pd.DataFrame({'MACD': macd, 'Signal': signal})


def calculate_macd_matrices(prices):
    """
    Calculate MACD and Signal Line indicators.
    
    Parameters:
    - prices (pd.Series): Series of prices.
    
    Returns:
    - Tuple[float, float]: MACD, Signal Line.
    """

    macd = compute_macd(prices, short_window=12, long_window=26, signal_window=9)

    # ta.trend.MACD(prices, window_fast=12, window_slow=26, window_sign=9)
    # macd, macd_signals = macd_series.macd(), macd_series.macd_signal()
    macd_returns = []
    buy_price = None

    for i in range(1, prices.shape[0]):
        if macd["MACD"].iloc[i] > macd["Signal"].iloc[i] and macd["MACD"].iloc[i - 1] <= macd["Signal"].iloc[i - 1]:
            buy_price = prices.iloc[i]
        elif macd["MACD"].iloc[i] < macd["Signal"].iloc[i] and macd["MACD"].iloc[i-1] >= macd["Signal"].iloc[i-1] and buy_price is not None:
            macd_returns.append((prices.iloc[i] - (buy_price * 1.00025) ) / buy_price)
            buy_price = 0
    macd_returns = np.array(macd_returns)
    try:
        if macd_returns.std() == 0:
            macd_sharpe_ratio = 0
        else:
            macd_sharpe_ratio = (macd_returns.sum() - 0.025) / macd_returns.std()
    except Exception as e:
        macd_sharpe_ratio = 0
    return macd_returns.sum(), macd_returns.std(), macd_sharpe_ratio

def calculate_rsi_matrices(prices):
    """
    Calculate RSI indicator.
    
    Parameters:
    - prices (pd.Series): Series of prices.
    
    Returns:
    - Tuple[float, float]: RSI, RSI Signal.
    """
    rsi_series = compute_rsi(prices, 14)
    rsi = rsi_series

    rsi_returns = []
    buy_price = None

    for i in range(prices.shape[0]):
        if rsi.iloc[i] < 30:
            buy_price = prices.iloc[i]
        elif rsi.iloc[i] > 70 and buy_price is not None:
            rsi_returns.append((prices.iloc[i] - (buy_price * 1.00025)) / buy_price)
            buy_price = None

    rsi_returns = np.array(rsi_returns)
    try:
        if rsi_returns.std() == 0:
            rsi_sharpe_ratio = 0
        else:
            rsi_sharpe_ratio = (rsi_returns.mean() - 0.025) / rsi_returns.std()
    except Exception as e:
        rsi_sharpe_ratio = 0
    return rsi_returns.mean(), rsi_returns.std(), rsi_sharpe_ratio

def calculate_bandh_matrices(prices):
    """
    Calculate Buy and Hold indicator.
    
    Parameters:
    - prices (pd.Series): Series of prices.
    
    Returns:
    - Tuple[float, float]: Buy and Hold, Buy and Hold Signal.
    """
    bandh_returns = []
    buy_price = None

    for index in range(prices.shape[0]):
        if index == 0:
            buy_price = prices.iloc[index]
        elif index == prices.shape[0] - 1 and buy_price is not None:
            bandh_returns.append((prices.iloc[index] - (buy_price * 1.00025) ) / buy_price)
            buy_price = 0
    bandh_returns = np.array(bandh_returns)
    return bandh_returns.mean()

# Define the tickers and download the data
tickers = ["ALL", "ASGN", "CI", "COP", "EME", "EVR", "GILD", "GPK", 
           "ISRG", "MKL", "MOH", "PEG", "PXD", "QCOM", "UBSI", "VFC", "XEL"]

# Download the adjusted close prices for the specified time period
start_date = "2017-11-27"
end_date = "2019-11-27"
data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

rors_macd = []
risks_macd = []
sharpe_ratios_macd = []

rors_rsi = []
risks_rsi = []
sharpe_ratios_rsi = []

rors_bandh = []

# Apply strategies and compute metrics
for ticker in tickers:
    prices = data[ticker]
    macd_return,macd_risk, macd_sharpe = calculate_macd_matrices(prices)
    rsi_return, rsi_risk, rsi_sharpe = calculate_rsi_matrices(prices)
    bandh_return = calculate_bandh_matrices(prices)
    rors_macd.append(macd_return)
    risks_macd.append(macd_risk)
    sharpe_ratios_macd.append(macd_sharpe)

    rors_rsi.append(rsi_return)
    risks_rsi.append(rsi_risk)
    sharpe_ratios_rsi.append(rsi_sharpe)

    rors_bandh.append(bandh_return)

    print(f"Metrics for {ticker}:")
    
    print(f"MACD - Rate of Return: {macd_return:.5f}, Risk: {macd_risk:.5f}, Sharpe Ratio: {macd_sharpe:.5f}")
    print(f"RSI - Rate of Return: {rsi_return:.5f}, Risk: {rsi_risk:.2f}, Sharpe Ratio: {rsi_sharpe:.5f}")
    print(f"Buy and Hold - Rate of Return: {bandh_return:.2f}")

print("-" * 50)
print("Average Metrics:")
print(f"MACD - Rate of Return: {np.array(rors_macd).mean():.5f}, Risk: {np.array(risks_macd).mean():.5f}, Sharpe Ratio: {np.array(sharpe_ratios_macd).mean():.5f}")
print(f"RSI - Rate of Return: {np.array(rors_rsi).mean():.5f}, Risk: {np.array(risks_rsi).mean():.2f}, Sharpe Ratio: {np.array(sharpe_ratios_rsi).mean():.5f}")
print(f"Buy and Hold - Rate of Return: {np.array(rors_bandh).mean():.2f}")

print("-" * 50)
print("Median Metrics:")
print(f"MACD - Rate of Return: {np.median(rors_macd):.5f}, Risk: {np.median(risks_macd):.5f}, Sharpe Ratio: {np.median(sharpe_ratios_macd):.5f}")
print(f"RSI - Rate of Return: {np.median(rors_rsi):.5f}, Risk: {np.median(risks_rsi):.2f}, Sharpe Ratio: {np.median(sharpe_ratios_rsi):.5f}")
print(f"Buy and Hold - Rate of Return: {np.median(rors_bandh):.2f}")

In [2]:

def split_func(df):
    # Define the split ratios
    train_ratio = 0.8

    # Calculate the split indices
    total_rows = len(df)
    train_split_idx = int(total_rows * train_ratio)

    # Split the data
    train_df = df.iloc[:train_split_idx].reset_index(drop=True)
    test_df = df.iloc[train_split_idx + 3:].reset_index(drop=True)

    return train_df, test_df


In [3]:
import pandas as pd
from strategy1 import load_strategy_1, strategy1_fitness_function

# Read the data from CSV
df = pd.read_csv("data/stock_data.csv")

train_df, test_df = split_func(df)
# Define thresholds
main_thresholds = (
    np.array([0.098, 0.22, 0.48, 0.72, 0.98, 1.22, 1.55, 1.70, 2, 2.55])
    / 100
)
for threshold in main_thresholds:
    # Load strategy 1 decisions
    stock_decision_by_threshold = load_strategy_1(
        df=test_df,
        thresholds=[threshold],
        pkl_filename=f"data/strategy1{threshold}_test_data.pkl",
    )

    RoR, volatility, sharpe_ratio = strategy1_fitness_function(
        test_df, [1], stock_decision_by_threshold
    )

    print(
        f"Strategy 1 matrices for threshold {threshold * 100}:"
    )
    print(f"RoR: {RoR:.5f}, Risk: {volatility:.5f}, Sharpe Ratio: {sharpe_ratio:.5f}")

Strategy 1 matrices for threshold 0.098:
RoR: 0.09034, Risk: 0.04821, Sharpe Ratio: 2.07627
Strategy 1 matrices for threshold 0.22:
RoR: 0.08632, Risk: 0.04609, Sharpe Ratio: 2.07168
Strategy 1 matrices for threshold 0.48:
RoR: 0.02298, Risk: 0.04836, Sharpe Ratio: 1.16396
Strategy 1 matrices for threshold 0.72:
RoR: 0.08096, Risk: 0.04847, Sharpe Ratio: 1.92047
Strategy 1 matrices for threshold 0.98:
RoR: 0.02613, Risk: 0.05143, Sharpe Ratio: 0.74171
Strategy 1 matrices for threshold 1.22:
RoR: -0.01482, Risk: 0.05441, Sharpe Ratio: 0.32935
Strategy 1 matrices for threshold 1.55:
RoR: -0.00793, Risk: 0.05801, Sharpe Ratio: 0.38233
Strategy 1 matrices for threshold 1.7000000000000002:
RoR: -0.02202, Risk: 0.06086, Sharpe Ratio: 0.55186
Strategy 1 matrices for threshold 2.0:
RoR: -0.02782, Risk: 0.06295, Sharpe Ratio: -0.02493
Strategy 1 matrices for threshold 2.55:
RoR: -0.03677, Risk: 0.06645, Sharpe Ratio: -0.37750


In [ ]:
from strategy2 import load_strategy_2, strategy2_fitness_function

# Read the data from CSV
df = pd.read_csv("data/stock_data.csv")

train_df, test_df = split_func(df)
# Define thresholds
main_thresholds = (
    np.array([0.098, 0.22, 0.48, 0.72, 0.98, 1.22, 1.55, 1.70, 2, 2.55])
    / 100
)
for threshold in main_thresholds:
    # Load strategy 1 decisions
    stock_decision_by_threshold = load_strategy_2(
        df=test_df,
        thresholds=[threshold],
        pkl_filename=f"data/strategy2{threshold}_test_data.pkl",
    )

    RoR, volatility, sharpe_ratio = strategy2_fitness_function(
        test_df, [1], stock_decision_by_threshold
    )

    print(
        f"Strategy 2 matrices for threshold {threshold * 100:.3f}:"
    )
    print(f"RoR: {RoR:.5f}, Risk: {volatility:.5f}, Sharpe Ratio: {sharpe_ratio:.5f}")

In [ ]:
from strategy3 import load_strategy_3, strategy3_fitness_function

# Read the data from CSV
df = pd.read_csv("data/stock_data.csv")

train_df, test_df = split_func(df)
# Define thresholds
main_thresholds = (
    np.array([0.098, 0.22, 0.48, 0.72, 0.98])
    / 100
)
for threshold in main_thresholds:
    # Load strategy 1 decisions
    stock_decision_by_threshold = load_strategy_3(
        df=test_df,
        thresholds=[threshold],
        pkl_filename=f"data/strategy3{threshold}_test_data.pkl",
    )

    RoR, volatility, sharpe_ratio = strategy3_fitness_function(
        test_df, [1], stock_decision_by_threshold
    )

    print(
        f"Strategy 3 matrices for threshold {threshold * 100:.3f}:"
    )
    print(f"RoR: {RoR:.5f}, Risk: {volatility:.5f}, Sharpe Ratio: {sharpe_ratio:.5f}")

In [ ]:
# Get the weights for strategy 1


# Initialize an empty list to store the weight arrays
weights_list_1 = []

# Open the text file for reading
with open('../results/02/output/strategy1_train_run.txt', 'r') as file:
    # Iterate over each line in the file
    for line in file:
        # Check if the line contains a weight array (using '[' as an indicator)
        if '[' in line:
            # Extract the weight array from the line
            weights = line.split('[')[1].split(']')[0]
            # Convert the weight array from string format to a list of floats
            weights = list(map(float, weights.split()))
            # Append the weight array to the weights_list
            weights_list_1.append(weights)

# Print the weights_list to verify the results
print(len(weights_list_1))


In [ ]:
# Get the weights for strategy 2


# Initialize an empty list to store the weight arrays
weights_list_2 = []

# Open the text file for reading
with open('../results/02/output/strategy2_train_run.txt', 'r') as file:
    # Iterate over each line in the file
    for line in file:
        # Check if the line contains a weight array (using '[' as an indicator)
        if '[' in line:
            # Extract the weight array from the line
            weights = line.split('[')[1].split(']')[0]
            # Convert the weight array from string format to a list of floats
            weights = list(map(float, weights.split()))
            # Append the weight array to the weights_list
            weights_list_2.append(weights)

# Print the weights_list to verify the results
print(len(weights_list_2))


In [ ]:
# Get the weights for strategy 3


# Initialize an empty list to store the weight arrays
weights_list_3 = []

# Open the text file for reading
with open('../results/02/output/strategy3_train_run.txt', 'r') as file:
    # Iterate over each line in the file
    for line in file:
        # Check if the line contains a weight array (using '[' as an indicator)
        if '[' in line:
            # Extract the weight array from the line
            weights = line.split('[')[1].split(']')[0]
            # Convert the weight array from string format to a list of floats
            weights = list(map(float, weights.split()))
            # Append the weight array to the weights_list
            weights_list_3.append(weights)

# Print the weights_list to verify the results
print(len(weights_list_3))


In [ ]:
for weights in weights_list_1:
    print(weights)

In [ ]:
weights_list_1_old = []
with open('../CEC/Strategy1/ALL/Results/BestModel.txt', 'r') as file:
    for line in file:
        # Split the line by tabs, skip the "Run" text, and convert the rest to floats
        floats = list(map(float, line.strip().split('\t')[1:]))
        weights_list_1_old.append(floats)

weights_list_1_old

In [ ]:
from strategy1 import load_strategy_1, strategy1_fitness_function

weights_list_1 = [[0.05929022, 0.185659,   0.04523656, 0.12992884, 0.08163506, 0.0446837, 0.24321252, 0.00423264, 0.00478092, 0.20134048]]

# Read the data from CSV
df = pd.read_csv("data/stock_data.csv")

train_df, test_df = split_func(df)

# Define thresholds
main_thresholds = (
    np.array([0.098, 0.22, 0.48, 0.72, 0.98, 1.22, 1.55, 1.70, 2, 2.55])
    / 100
)

for weights in weights_list_1:
    # Load strategy 1 decisions
    stock_decision_by_threshold = load_strategy_1(
        df=test_df,
        thresholds=main_thresholds,
        pkl_filename=f"data/strategy1_test_data.pkl",
        excel_filename=f"output/strategy1_test_data.xlsx",
        export_excel=True,
    )

    RoR, volatility, sharpe_ratio = strategy1_fitness_function(
        test_df, weights, stock_decision_by_threshold
    )
    print(
        f"Strategy 1 matrices:"
    )
    print(f"RoR: {RoR:.5f}, Risk: {volatility:.5f}, Sharpe Ratio: {sharpe_ratio:.5f}")
    print('-' * 50)

In [ ]:
from strategy2 import load_strategy_2, strategy2_fitness_function

# Read the data from CSV
df = pd.read_csv("data/stock_data.csv")

train_df, test_df = split_func(df)
# Define thresholds
main_thresholds = (
    np.array([0.098, 0.22, 0.48, 0.72, 0.98, 1.22, 1.55, 1.70, 2, 2.55])
    / 100
)
for weights in weights_list_2:
    # Load strategy 1 decisions
    stock_decision_by_threshold = load_strategy_2(
        df=test_df,
        thresholds=main_thresholds,
        pkl_filename=f"data/strategy2_test_data.pkl",
    )

    RoR, volatility, sharpe_ratio = strategy2_fitness_function(
        test_df, weights, stock_decision_by_threshold
    )
    print(
        f"Strategy 2 matrices:"
    )
    print(f"RoR: {RoR:.5f}, Risk: {volatility:.5f}, Sharpe Ratio: {sharpe_ratio:.5f}")
    print('-' * 50)

In [ ]:
from strategy3 import load_strategy_3, strategy3_fitness_function

# Read the data from CSV
df = pd.read_csv("data/stock_data.csv")

train_df, test_df = split_func(df)
# Define thresholds
main_thresholds = (
    np.array([0.098, 0.22, 0.48, 0.72, 0.98])
    / 100
)
for weights in weights_list_3:
    # Load strategy 1 decisions
    stock_decision_by_threshold = load_strategy_3(
        df=test_df,
        thresholds=main_thresholds,
        pkl_filename=f"data/strategy3_test_data.pkl",
    )

    RoR, volatility, sharpe_ratio = strategy3_fitness_function(
        test_df, weights, stock_decision_by_threshold
    )
    print(
        f"Strategy 3 matrices:"
    )
    print(f"Weights: {weights}")
    print(f"RoR: {RoR:.5f}, Risk: {volatility:.5f}, Sharpe Ratio: {sharpe_ratio:.5f}")
    print('-' * 50)

In [ ]:
import random

def weighted_random_choice(choices):
    total_weight = sum(weight for choice, weight in choices.items())
    random_num = random.randint(1, total_weight)
    
    for choice, weight in choices.items():
        random_num -= weight
        if random_num <= 0:
            return choice

# Example usage:
choices = {"A": 5, "B": 3, "C": 2}
print(weighted_random_choice(choices))

In [ ]:
str(tickers)